Original yolov5 TL article: https://kikaben.com/yolov5-transfer-learning-dogs-cats/
Yolov5 mask-wearing: https://towardsdatascience.com/mask-detection-using-yolov5-ae40979227a6

Yolov5 model github: https://github.com/ultralytics/yolov5

Yolov5 transfer learning doc: https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#11-create-datasetyaml


Yolov5 instance segmentation TL: https://learnopencv.com/yolov5-instance-segmentation/
Yolov5 github segmentation section: https://github.com/ultralytics/yolov5#segmentation

In [1]:
from PIL import Image, ImageDraw
import os
import glob
import numpy as np
import shutil

In [2]:
#read in all item classes
class_file = open("oi_schoolimgs/darknet_obj_names.txt", "r")
classes = class_file.read()
classes = classes.split('\n') #convert to list
classes = classes[:-1] #drop empty last element
classes

['backpack',
 'scissors',
 'calculator',
 'stapler',
 'eraser',
 'adhesive tape',
 'pencil case',
 'ruler',
 'pen',
 'envelope',
 'book',
 'laptop']

In [ ]:
#Data pre-processing
# Final??
def show_bbox(image_path):
    # convert image path to label path
    label_path = image_path.replace('/images/', '/darknet/')
    label_path = label_path.replace('.jpg', '.txt')

    # Open the image and create ImageDraw object for drawing
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    with open(label_path, 'r') as f:
        for line in f.readlines():
            # Split the line into five values
            label, x, y, w, h = line.split(' ')

            # Convert string into float
            x = float(x)
            y = float(y)
            w = float(w)
            h = float(h)

            # Convert center position, width, height into
            # top-left and bottom-right coordinates
            W, H = image.size
            x1 = (x - w/2) * W
            y1 = (y - h/2) * H
            x2 = (x + w/2) * W
            y2 = (y + h/2) * H

            # Draw the bounding box with red lines
            draw.rectangle((x1, y1, x2, y2),
                           outline=(255, 0, 0), # Red in RGB
                           width=5)             # Line width
    image.show()
    

#Test display an image with BBox(es)
show_bbox('oi_schoolimgs/images/train/0a0df46ca3f886c9.jpg')


In [3]:
#Re-structure data folders

# Create a folder structure for YOLOv5 training
if not os.path.exists('data'):
    for folder in ['images', 'labels']:
        for split in ['train', 'val', 'test']:
            os.makedirs(f'data/{folder}/{split}')



In [4]:
#Check for duplicates in same downloaded folder
def get_filenames(folder):
    filenames = set()
    
    for path in glob.glob(os.path.join(folder, '*.jpg')):
        # Extract the filename
        filename = os.path.split(path)[-1]        
        filenames.add(filename)

    return filenames


classes_imgs_dict = dict()
for cls in classes:
    classes_imgs_dict[cls] = get_filenames(f"oi_schoolimgs/{cls}/images")

imgs_cnts = dict()
for cls in classes_imgs_dict.keys():
    for img in classes_imgs_dict[cls]:
        if img not in imgs_cnts.keys():
            imgs_cnts[img] = (1, [cls])
        else:
            imgs_cnts[img] = (imgs_cnts[img][0]+1, imgs_cnts[img][1]+[cls])


In [5]:
# Check for duplicates
[i for i in imgs_cnts.items() if i[1][0]>1]

[('008b7daf18c2bd65.jpg', (2, ['pen', 'laptop']))]

In [6]:
#Drop duplicates
classes_imgs_dict['pen'] -= {'008b7daf18c2bd65.jpg'}
classes_imgs_dict['laptop'] -= {'008b7daf18c2bd65.jpg'}

In [7]:
#Shuffle randomly all the files (with fixed seed)
np.random.seed(1337)

for cls in classes_imgs_dict.keys():
    temp_ary = np.array(list(classes_imgs_dict[cls]))
    np.random.shuffle(temp_ary)
    temp_ary
    classes_imgs_dict[cls] = temp_ary


In [8]:
#Copy image files to new directory structure for Transfer Learning (in YOLO expected structure)
def split_dataset(cls, train_pct=0.8, val_pct=0.1, download_dir='oi_schoolimgs'):
    cls_imgs = classes_imgs_dict[cls]
    cls_img_cnt = len(cls_imgs)
    
    train_size = round(train_pct * cls_img_cnt)
    val_size = round(val_pct * cls_img_cnt)

    for num, img_nm in enumerate(cls_imgs):
        lbl_nm = img_nm.replace('.jpg', '.txt')
        
        # Split into train, val, or test
        if num < train_size:
            split = 'train'
        elif num < train_size + val_size:
            split = 'val'
        else:
            split = 'test'
        
        # Source paths
        source_image_path = f'{download_dir}/{cls}/images/{img_nm}'
        source_label_path = f'{download_dir}/{cls}/darknet/{lbl_nm}'

        # Destination paths
        target_image_folder = f'data/images/{split}'
        target_label_folder = f'data/labels/{split}'

        # Copy files
        shutil.copy(source_image_path, target_image_folder)
        shutil.copy(source_label_path, target_label_folder)

#Call function for each class
for c in classes:
    split_dataset(c)